In [1]:
### 한국어 사용하는 LLM 사용해보기 on cpu (gpu memory 6기가밖에 안되가지공,,)

from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
model = AutoModelForCausalLM.from_pretrained('microsoft/DialoGPT-small').to('cpu')

In [2]:
new_user_input_ids = tokenizer.encode("한국말해봐" + tokenizer.eos_token,return_tensors='pt')
bot_input_ids = new_user_input_ids
result_chat = model.generate(bot_input_ids,max_length = 300, pad_token_id = tokenizer.eos_token_id)

print("DialoGPT: {}".format(tokenizer.decode(result_chat[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm not sure if you're joking or not, but I'm pretty sure it's a joke.


In [3]:
import datasets

dataset = datasets.load_dataset('jiwoochris/easylaw_kr')

In [4]:
print(dataset)
#instruction : 질문
#output : 답변 


DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'input'],
        num_rows: 2195
    })
})


In [5]:
from peft import LoraConfig,get_peft_model

lora_r = 8 #rank setting
lora_alpha = 32 #제한
#lora_dropout = 0.05 #dropout설정
lora_target_modules = ['c_attn','c_proj']
print(model.modules)
config = LoraConfig(
    r = lora_r,
    lora_alpha= lora_alpha,
    target_modules= lora_target_modules,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model,config)

<bound method Module.modules of GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)>


c:\Users\HOME\anaconda3\lib\site-packages\peft\tuners\lora\layer.py:1059: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [6]:
train_data = dataset


In [7]:
print(train_data)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'input'],
        num_rows: 2195
    })
})


In [13]:
num_epochs = 5
batch_size = 2

learning_rate = 1e-05
gradient_accumulation_steps = 2
output_dir = './output'
world_size = 1
ddp = world_size != 1
group_by_length = False
val_set_size = 0
import transformers
train_args = transformers.TrainingArguments(
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    warmup_steps=100,
    num_train_epochs=num_epochs,
    learning_rate=learning_rate,
    bf16=False,
    logging_steps=2,
    evaluation_strategy="steps" if val_set_size > 0 else "no",
    eval_steps=200 if val_set_size > 0 else None,
    output_dir=output_dir,
    load_best_model_at_end=True if val_set_size > 0 else False,
    ddp_find_unused_parameters=False if ddp else None,
    report_to="none",
    group_by_length=group_by_length,
)

In [14]:
"""data_collator = transformers.DataCollatorForSeq2Seq(tokenizer, return_tensors="pt", padding=True)
class TokenizerWrapper:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
    
    def tokenize_function(self, examples):
        return self.tokenizer(
            examples['instruction'],
            padding="max_length",
            truncation=True,
        ) 
    def tokenize_function_labels(self, examples):
        return self.tokenizer(
            examples['output'],
            padding="max_length",
            truncation=True,
        ) 
special_tokens = tokenizer.special_tokens_map
print(special_tokens)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
def add_end_token_to_question(input_dict):
    input_dict['instruction'] += special_tokens['bos_token']
    input_dict['output'] += special_tokens['bos_token']
    return input_dict

train_data = train_data.map(add_end_token_to_question)
tokenizer_wrapper = TokenizerWrapper(tokenizer)

tokenized_dataset = train_data.map(tokenizer_wrapper.tokenize_function, batched=True, num_proc=4, remove_columns=['input'])
tokenized_label = train_data.map(tokenizer_wrapper.tokenize_function, batched=True, num_proc=4, remove_columns=['input'])
model.resize_token_embeddings(len(tokenizer))
"""

special_tokens = tokenizer.special_tokens_map
print(special_tokens)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
def add_end_token_to_question(input_dict):
    input_dict['instruction'] += special_tokens['bos_token']
    input_dict['output'] += special_tokens['bos_token']
    return input_dict

train_data = train_data.map(add_end_token_to_question)
def add_end_token_to_question(input_dict):
    input_dict['instruction'] += special_tokens['bos_token']
    input_dict['output'] += special_tokens['bos_token']
    return input_dict

train_data = train_data.map(add_end_token_to_question)
model.resize_token_embeddings(len(tokenizer))

{'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '[PAD]'}


Embedding(50258, 768)

In [15]:
from functools import partial
import copy
from transformers import DataCollatorForSeq2Seq
from typing import Dict, List

MAX_LENGTH = 256

# Function to generate token embeddings from text part of batch
def _preprocess_batch(batch: Dict[str, List]):  
    model_inputs = tokenizer(batch["instruction"], max_length=MAX_LENGTH, truncation=True, padding='max_length')    
    model_inputs["labels"] = copy.deepcopy(model_inputs['input_ids'])
    return model_inputs

_preprocessing_function = partial(_preprocess_batch)

# apply the preprocessing function to each batch in the dataset
encoded_small_dataset = train_data['train'].map(
        _preprocessing_function,
        batched=True,
        remove_columns=["instruction", "output"],
)
processed_dataset = encoded_small_dataset.filter(lambda rec: len(rec["input_ids"]) <= MAX_LENGTH)

# splitting dataset
split_dataset = processed_dataset.train_test_split(test_size=14, seed=0)
print(split_dataset)

# takes a list of samples from a Dataset and collate them into a batch, as a dictionary of PyTorch tensors.
data_collator = DataCollatorForSeq2Seq(
        model = model, tokenizer=tokenizer, max_length=MAX_LENGTH, pad_to_multiple_of=8, padding='max_length')

Map:   0%|          | 0/2195 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2195 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2181
    })
    test: Dataset({
        features: ['input', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14
    })
})


In [16]:
trainer = transformers.Trainer(
    model = model,
    train_dataset=split_dataset['train'],
    args = train_args,
    data_collator=data_collator
)

c:\Users\HOME\anaconda3\lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [17]:
trainer.train()

  0%|          | 0/2725 [00:00<?, ?it/s]

{'loss': 148.9142, 'learning_rate': 2.0000000000000002e-07, 'epoch': 0.0}
{'loss': 120.6206, 'learning_rate': 4.0000000000000003e-07, 'epoch': 0.01}
{'loss': 209.8717, 'learning_rate': 6.000000000000001e-07, 'epoch': 0.01}
{'loss': 164.6684, 'learning_rate': 8.000000000000001e-07, 'epoch': 0.01}
{'loss': 165.9668, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.02}
{'loss': 216.1205, 'learning_rate': 1.2000000000000002e-06, 'epoch': 0.02}
{'loss': 181.2519, 'learning_rate': 1.4000000000000001e-06, 'epoch': 0.03}
{'loss': 225.7728, 'learning_rate': 1.6000000000000001e-06, 'epoch': 0.03}
{'loss': 163.6481, 'learning_rate': 1.8000000000000001e-06, 'epoch': 0.03}
{'loss': 143.5117, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.04}
{'loss': 172.7267, 'learning_rate': 2.2e-06, 'epoch': 0.04}
{'loss': 203.627, 'learning_rate': 2.4000000000000003e-06, 'epoch': 0.04}
{'loss': 212.5016, 'learning_rate': 2.6e-06, 'epoch': 0.05}
{'loss': 136.0235, 'learning_rate': 2.8000000000000003e-06,

c:\Users\HOME\anaconda3\lib\site-packages\peft\utils\save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


{'loss': 2.453, 'learning_rate': 6.563809523809524e-06, 'epoch': 1.84}
{'loss': 2.266, 'learning_rate': 6.556190476190477e-06, 'epoch': 1.84}
{'loss': 2.8717, 'learning_rate': 6.548571428571428e-06, 'epoch': 1.84}
{'loss': 2.9585, 'learning_rate': 6.540952380952381e-06, 'epoch': 1.85}
{'loss': 2.9462, 'learning_rate': 6.533333333333334e-06, 'epoch': 1.85}
{'loss': 2.3393, 'learning_rate': 6.525714285714286e-06, 'epoch': 1.86}
{'loss': 1.9945, 'learning_rate': 6.518095238095239e-06, 'epoch': 1.86}
{'loss': 2.741, 'learning_rate': 6.510476190476192e-06, 'epoch': 1.86}
{'loss': 2.1419, 'learning_rate': 6.502857142857143e-06, 'epoch': 1.87}
{'loss': 2.4419, 'learning_rate': 6.495238095238096e-06, 'epoch': 1.87}
{'loss': 2.4509, 'learning_rate': 6.487619047619048e-06, 'epoch': 1.87}
{'loss': 2.4911, 'learning_rate': 6.480000000000001e-06, 'epoch': 1.88}
{'loss': 2.1605, 'learning_rate': 6.4723809523809535e-06, 'epoch': 1.88}
{'loss': 2.5204, 'learning_rate': 6.464761904761905e-06, 'epoch': 

c:\Users\HOME\anaconda3\lib\site-packages\peft\utils\save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


...

c:\Users\HOME\anaconda3\lib\site-packages\peft\utils\save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


{'loss': 2.0897, 'learning_rate': 2.7542857142857145e-06, 'epoch': 3.67}
{'loss': 1.8044, 'learning_rate': 2.746666666666667e-06, 'epoch': 3.67}
{'loss': 1.9842, 'learning_rate': 2.7390476190476193e-06, 'epoch': 3.68}
{'loss': 2.3677, 'learning_rate': 2.7314285714285714e-06, 'epoch': 3.68}
{'loss': 1.8572, 'learning_rate': 2.723809523809524e-06, 'epoch': 3.68}
{'loss': 2.1046, 'learning_rate': 2.7161904761904762e-06, 'epoch': 3.69}
{'loss': 2.5128, 'learning_rate': 2.7085714285714287e-06, 'epoch': 3.69}
{'loss': 2.2816, 'learning_rate': 2.7009523809523815e-06, 'epoch': 3.7}
{'loss': 1.9353, 'learning_rate': 2.6933333333333335e-06, 'epoch': 3.7}
{'loss': 2.2687, 'learning_rate': 2.685714285714286e-06, 'epoch': 3.7}
{'loss': 2.1108, 'learning_rate': 2.6780952380952384e-06, 'epoch': 3.71}
{'loss': 1.7277, 'learning_rate': 2.6704761904761904e-06, 'epoch': 3.71}
{'loss': 1.9568, 'learning_rate': 2.6628571428571433e-06, 'epoch': 3.71}
{'loss': 1.7718, 'learning_rate': 2.6552380952380957e-06,

c:\Users\HOME\anaconda3\lib\site-packages\peft\utils\save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


{'loss': 2.7909, 'learning_rate': 8.495238095238096e-07, 'epoch': 4.59}
{'loss': 2.0242, 'learning_rate': 8.41904761904762e-07, 'epoch': 4.59}
{'loss': 1.7513, 'learning_rate': 8.342857142857144e-07, 'epoch': 4.59}
{'loss': 1.8325, 'learning_rate': 8.266666666666668e-07, 'epoch': 4.6}
{'loss': 1.7371, 'learning_rate': 8.190476190476192e-07, 'epoch': 4.6}
{'loss': 1.7366, 'learning_rate': 8.114285714285715e-07, 'epoch': 4.6}
{'loss': 1.8186, 'learning_rate': 8.03809523809524e-07, 'epoch': 4.61}
{'loss': 2.2758, 'learning_rate': 7.961904761904763e-07, 'epoch': 4.61}
{'loss': 1.9375, 'learning_rate': 7.885714285714287e-07, 'epoch': 4.62}
{'loss': 1.7738, 'learning_rate': 7.809523809523809e-07, 'epoch': 4.62}
{'loss': 2.1634, 'learning_rate': 7.733333333333335e-07, 'epoch': 4.62}
{'loss': 1.8518, 'learning_rate': 7.657142857142857e-07, 'epoch': 4.63}
{'loss': 2.3551, 'learning_rate': 7.580952380952381e-07, 'epoch': 4.63}
{'loss': 1.881, 'learning_rate': 7.504761904761905e-07, 'epoch': 4.63

TrainOutput(global_step=2725, training_loss=17.079943937774097, metrics={'train_runtime': 970.385, 'train_samples_per_second': 11.238, 'train_steps_per_second': 2.808, 'train_loss': 17.079943937774097, 'epoch': 5.0})

In [51]:
infer_prompt = "한국어할줄 알아?"
new_user_input_ids = tokenizer.encode(tokenizer.eos_token+infer_prompt,return_tensors='pt')
print(new_user_input_ids)
bot_input_ids = new_user_input_ids
result_chat = trainer.model.generate(bot_input_ids,max_length = 500, pad_token_id = tokenizer.eos_token_id)
print(result_chat)
print("DialoGPT: {}".format(tokenizer.decode(result_chat[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

tensor([[50256, 47991,   250,   166,   113,   255,   168,   244,   112, 47991,
           254,   168,    97,   226, 23821,   243,   234,   168,   243,   226,
            30]])
tensor([[50256, 47991,   250,   166,   113,   255,   168,   244,   112, 47991,
           254,   168,    97,   226, 23821,   243,   234,   168,   243,   226,
            30,   168,   226,   168,   243,   226,   168,   226,   168,   244,
           112,   168,   244,   112,   168,   226,   168,   245,   168,   250,
           222,   168,   244,   112,   168,   226,   226, 23821,   254,   168,
           244,   168,   244,   112,   168,   226,   168,   244,   226,   168,
           246,   168,   244,   168,   244,   112,   168,   226,   250, 23821,
           244,   112,   168,   226, 47991,   246,   168,   244,   112, 23821,
           244,   112,   168,   246,   168,   244,   112,   220,   166,   112,
           112,   168,   226,   168,   245,   168,   226,   168,   248,   112,
           168,   226,   168,   24

In [68]:
infer_prompt = "교통법규 위반"
new_user_input_ids = tokenizer.encode(tokenizer.eos_token+infer_prompt,return_tensors='pt')
bot_input_ids = new_user_input_ids
result_chat = model.generate(bot_input_ids,max_length = 256, pad_token_id = tokenizer.eos_token_id)
print("DialoGPT: {}".format(tokenizer.decode(result_chat[0], skip_special_tokens=True)))

DialoGPT: 교통법규 위반곴�볘 였�늀곘 옼��곘늀곘 점곘디볘 � � 였� 옘�늀 겘옘렴늀� � �곘� � 였�늘�� 였�덀�던딘예�� �늘���늘늘�늘렀���늘 ���던던는� � �늘� � �늘 �늘늘�덍늘�늘덐��던��


In [74]:
print("DialoGPT: {}".format(tokenizer.decode(result_chat[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

TypeError: 'NoneType' object is not callable